In [2]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


In [3]:
# Configuration
data_dir = '/kaggle/input/face-classification-deep-learning-cs-405/dataset'  # Replace with your actual path
val_ratio = 0.1
test_ratio = 0.1
batch_size = 64
image_size = 224


In [4]:
# Train transforms with augmentations
train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Evaluation transforms (no augmentations)
eval_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [5]:
# Load full dataset with training transform (we'll change it later for val/test)
full_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'),
                                    transform=train_transform)

class_names = full_dataset.classes
total_len = len(full_dataset)
print(f"Total images: {total_len}")


Total images: 140000


In [6]:
val_len = int(total_len * val_ratio)
test_len = int(total_len * test_ratio)
train_len = total_len - val_len - test_len

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_len, val_len, test_len]
)


In [7]:
# Override transform for val and test datasets
val_dataset.dataset.transform = eval_transform
test_dataset.dataset.transform = eval_transform


In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=4, pin_memory=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=4, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=4, pin_memory=True)


In [9]:
print(f"[INFO] Train: {train_len}, Val: {val_len}, Test: {test_len}, Classes: {len(class_names)}")
print("Class Names:", class_names)


[INFO] Train: 112000, Val: 14000, Test: 14000, Classes: 7000
Class Names: ['n000001', 'n000002', 'n000003', 'n000004', 'n000005', 'n000008', 'n000011', 'n000012', 'n000013', 'n000014', 'n000015', 'n000016', 'n000017', 'n000018', 'n000019', 'n000020', 'n000021', 'n000022', 'n000023', 'n000024', 'n000025', 'n000026', 'n000027', 'n000029', 'n000030', 'n000031', 'n000034', 'n000035', 'n000036', 'n000037', 'n000038', 'n000040', 'n000041', 'n000042', 'n000043', 'n000044', 'n000045', 'n000046', 'n000047', 'n000048', 'n000049', 'n000050', 'n000051', 'n000052', 'n000053', 'n000054', 'n000055', 'n000056', 'n000057', 'n000058', 'n000059', 'n000060', 'n000061', 'n000062', 'n000063', 'n000066', 'n000067', 'n000068', 'n000069', 'n000070', 'n000071', 'n000074', 'n000075', 'n000077', 'n000079', 'n000082', 'n000084', 'n000085', 'n000086', 'n000088', 'n000089', 'n000090', 'n000091', 'n000092', 'n000093', 'n000094', 'n000095', 'n000096', 'n000097', 'n000098', 'n000100', 'n000101', 'n000106', 'n000108', '

In [10]:
import torch
import torch.nn as nn
from torchvision import models

# Configuration
num_classes = len(class_names)

# Load pretrained MobileNetV2
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier to match the number of classes in your dataset
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Print model summary
print(model)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 116MB/s] 


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [11]:
def save_checkpoint(model, optimizer, epoch, loss, path="checkpoint.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }, path)


In [ ]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training Loop
num_epochs = 20  # Change as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for inputs, labels in train_loader_tqdm:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        train_loader_tqdm.set_postfix(loss=loss.item(), acc=100 * correct / total)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    # Epoch Summary
    avg_train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    checkpoint_path = f"checkpoint_epoch_{epoch+1}.pth"
    save_checkpoint(model, optimizer, epoch+1, avg_val_loss, checkpoint_path)
    print(f"[INFO] Checkpoint saved at {checkpoint_path}")





Epoch 1/20:  25%|██▌       | 445/1750 [01:43<05:12,  4.18it/s, acc=0.014, loss=8.84]  

In [ ]:
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Final test accuracy
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")
